In [ ]:
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt

In [ ]:
def get_model(optimizer, loss, metrics):
    input_ = tf.keras.layers.Input(shape=(224, 224, 1), name="INPUT")
    #normalization = tf.keras.layers.Rescaling(1./255)(input_)
    n1 = tf.keras.layers.BatchNormalization()(input_)
    conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(7, 7), strides=1, activation="relu", padding="same", name="CONV1", data_format='channels_last', kernel_regularizer=tf.keras.regularizers.l2(0.01))(n1) 
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2, name="POOL1")(conv1) 
    dropout0 = tf.keras.layers.Dropout(0.5)(pool1)

    conv2 = tf.keras.layers.Conv2D(filters=32, kernel_size=(7, 7), strides=1, activation="relu", padding="same", name="CONV2")(dropout0)
    n2 = tf.keras.layers.BatchNormalization()(conv2)
    concat1 = tf.keras.layers.Concatenate()([pool1, n2])
    
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2, name="POOL2")(concat1)
    dropout01 = tf.keras.layers.Dropout(0.5)(pool2)
    conv3 = tf.keras.layers.Conv2D(filters=64, kernel_size=(5, 5), strides=1, activation="relu", padding="same", name="CONV3")(dropout01)
    n3 = tf.keras.layers.BatchNormalization()(conv3)
    concat2 = tf.keras.layers.Concatenate()([pool2, n3])
    
    pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2, name="POOL3")(concat2)
    dropout02 = tf.keras.layers.Dropout(0.5)(pool3)
    conv4 = tf.keras.layers.Conv2D(filters=64, kernel_size=(5, 5), strides=1, activation="relu", padding="same", name="CONV4")(dropout02)
    n4 = tf.keras.layers.BatchNormalization()(conv4)
    concat3 = tf.keras.layers.Concatenate()([pool3, n4])
    
    pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2, name="POOL4")(concat3)
    dropout03 = tf.keras.layers.Dropout(0.5)(pool4)

    flatten = tf.keras.layers.Flatten()(dropout03)
    
    fc1 = tf.keras.layers.Dense(4096, activation='relu')(flatten)
    fc2 = tf.keras.layers.Dense(2048, activation='relu')(fc1)
    
    output = tf.keras.layers.Dense(7, activation='softmax')(fc2)
    
    model = tf.keras.Model(
        inputs=[input_],
        outputs=[output]
    )

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

In [ ]:
train_dir = pathlib.Path("/dbfs/FileStore/tables/Train").with_suffix('')

In [ ]:
im_size = (224, 224)
val_split = 0.1
batch_size = 64
seed = 13
mode = "grayscale"

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=val_split,
    subset="training",
    seed=seed,
    image_size=(im_size),
    batch_size=batch_size,
    color_mode=mode
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=val_split,
    subset="validation",
    seed=seed,
    image_size=(im_size),
    batch_size=batch_size,
    color_mode=mode
)

In [ ]:
model = get_model(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                  metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_ds,
    epochs=10,
    batch_size=64,
    validation_data=(val_ds)
)

In [ ]:
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

epochs = range(1, len(training_loss) + 1)

plt.plot(epochs, training_loss, 'r-')
plt.plot(epochs, validation_loss, 'b-')
plt.legend(['Training Loss', 'Validation Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Epochs vs Loss')
plt.show()

In [ ]:
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

plt.plot(epochs, training_accuracy, 'r-')
plt.plot(epochs, validation_accuracy, 'b-')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Epochs vs Accuracy')
plt.show()